In [1]:
import random
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [2]:
# functions
def add_address_of_data(given_address): # could be useful for easily allowing others to use this file
    return "".join([given_address, "/visit_meaning_vectors/visit_meanings.csv"])

# calculating Euclidean distance
def calculate_prob_of_visit_with_sorting(given_visit, given_pca):
    # assuming given visit is a meaning vector, with number of variables same as number of PCs

    given_pca_sorted = given_pca.sort_values(by=["Labels"]) # sorting given PCA list by labels
    prev_label = 0
    current_label = 0
    lowest_dist_to_visit = 0
    current_labels_dist = 0
    total_dist = 0
    lowest_dist_to_visits_label = 0

    for x in range(0, len(given_pca_sorted)):
        current_label = given_pca_sorted.at[given_pca_sorted.index[x], "Labels"] # get current label

        # extracting required variables of the point in PCA data
        temp_point = given_pca_sorted.iloc[x]
        temp_point = temp_point.tolist()
        temp_point = temp_point[:-1]

        if current_label >= 0: # to skip "-1" labels
            if current_label == prev_label: current_labels_dist += np.linalg.norm(np.array(given_visit) - np.array(temp_point)) # performing the euclidean distance calculation
            else: # have arrived to next label in the sorted PCA, so check final values
                if current_labels_dist < lowest_dist_to_visit:
                    lowest_dist_to_visit = current_labels_dist
                    lowest_dist_to_visits_label = current_label
                prev_label = current_label
                total_dist += current_labels_dist
                current_labels_dist = 0 # since we are going to next label, reset distance

    if total_dist == 0: print(len(given_visit))
    return 1 - (lowest_dist_to_visits_label / total_dist), lowest_dist_to_visits_label

# calculating Euclidean distance and assumes that "given_pca" is already sorted
def calculate_prob_of_visit_without_sorting(given_visit, given_pca_sorted):
    # assuming given visit is a meaning vector, with number of variables same as number of PCs

    prev_label = 0
    current_label = 0
    lowest_dist_to_visit = 0
    current_labels_dist = 0
    total_dist = 0
    lowest_dist_to_visits_label = 0

    for x in range(0, len(given_pca_sorted)):
        current_label = given_pca_sorted.at[given_pca_sorted.index[x], "Labels"] # get current label

        # extracting required variables of the point in PCA data
        temp_point = given_pca_sorted.iloc[x]
        temp_point = temp_point.tolist()
        temp_point = temp_point[:-1]

        if current_label >= 0: # to skip "-1" labels
            if current_label == prev_label: current_labels_dist += np.linalg.norm(np.array(given_visit) - np.array(temp_point)) # performing the euclidean distance calculation
            else: # have arrived to next label in the sorted PCA, so check final values
                if current_labels_dist < lowest_dist_to_visit:
                    lowest_dist_to_visit = current_labels_dist
                    lowest_dist_to_visits_label = current_label
                prev_label = current_label
                total_dist += current_labels_dist
                current_labels_dist = 0 # since we are going to next label, reset distance

    if total_dist == 0: print(len(given_visit))
    return 1 - (lowest_dist_to_visits_label / total_dist), lowest_dist_to_visits_label

def get_avg_pca(given_pca):
    given_pca_sorted = given_pca.sort_values(by=["Labels"]) # sorting given PCA list by labels
    list_of_avg_values = []
    prev_label = 0
    current_labels_dist = [0] * (len(given_pca.columns) - 1)
    current_label_count = 0
    for x in range(0, len(given_pca_sorted)):
        current_label = given_pca_sorted.at[given_pca_sorted.index[x], "Labels"] # get current label


        if current_label >= 0: # to skip "-1" labels
            current_label_count += 1

            if current_label == prev_label:
                # extracting required variables of the point in PCA data
                temp_point = given_pca_sorted.iloc[x]
                temp_point = temp_point.tolist()
                temp_point = temp_point[:-1]

                current_labels_dist = [current_labels_dist[y] + temp_point[y] for y in range (0, len(current_labels_dist))]
            else: # have arrived to next label in the sorted PCA, so check final values
                prev_label = current_label
                list_of_avg_values.append([current_labels_dist[z] / current_label_count for z in range (0, len(current_labels_dist))])
                # reseting
                current_labels_dist = [0] * (len(given_pca.columns)-1)
                current_label_count = 0

    temp_df = pd.DataFrame(list_of_avg_values, columns=[b for b in range(0, len(given_pca.columns)-1)])
    return temp_df

# calculating Euclidean distance and assumes that "given_pca" is already sorted and averaged
def calculate_prob_of_visit_with_sorted_avg_pca(given_visit, given_avg_pca_sorted):
    # assuming given visit is a meaning vector, with number of variables same as number of PCs

    lowest_dist_to_visit = 0
    current_labels_dist = 0
    total_dist = 0
    lowest_dist_to_visits_label = 0

    for x in range(0, len(given_avg_pca_sorted)): # each index is a label
        # extracting required variables of the point in PCA data
        temp_point = given_avg_pca_sorted.iloc[x]
        temp_point = temp_point.tolist()
        temp_point = temp_point[:-1]

        current_labels_dist += np.linalg.norm(np.array(given_visit) - np.array(temp_point)) # performing the euclidean distance calculation

        if current_labels_dist < lowest_dist_to_visit:
            lowest_dist_to_visit = current_labels_dist
            lowest_dist_to_visits_label = x

        total_dist += current_labels_dist
        current_labels_dist = 0 # since we are going to next label, reset distance

    if total_dist == 0: print(len(given_visit))
    return 1 - (lowest_dist_to_visits_label / total_dist), lowest_dist_to_visits_label

In [3]:
# meaning vectors will be made of several hundreds of variables and each cluster will have a set of data points (each data point containing values of each of these variables (so a meaning vector)

# main data
my_address = "C:/Users/dnaen/APG_data"  # only this has to be modified
df = pd.read_csv(add_address_of_data(my_address))

ParserError: Error tokenizing data. C error: out of memory

In [ ]:
# will be used to get the labels
combined_address = "".join([my_address, "/cluster_paths.csv"])
df_with_labels = pd.read_csv(combined_address)

df_with_labels.head()

In [ ]:
# dividing the dataset into a feature set and corresponding labels
features_X = df.pop("meaning_vectors")
labels_Y = df_with_labels.pop("cluster_label")


In [ ]:
features_X = features_X.to_frame(name="meaning_vectors") # converting from series to df

In [ ]:
# expanding features df, because currently thes is only one column where each row contains a list of meaning values, to make it work for PCA method defined in sklearn library, it needs to be a df where each cell is only one number
features_X_expanded = features_X.meaning_vectors.str.split(",",expand=True,)
features_X_expanded.head(10)

In [ ]:
len(features_X_expanded.columns)

In [ ]:
# since we have expanded a string of list, we also have to remov "[" and "]" from first and last column
fixed_first_column = features_X_expanded[0].str.replace("[","")
fixed_last_column = features_X_expanded[99].str.replace("]","")

features_X_expanded[0] = fixed_first_column
features_X_expanded[99] = fixed_last_column

features_X_expanded.head()

In [ ]:
# Perform PCA on all data
# first create train and test
features_X_expanded["Labels"] = labels_Y # appending labels to not lose their assigned labels when performing data split
train, test = train_test_split(features_X_expanded, test_size=0.2)

# saving labels
train_labels = train.pop("Labels")
features_X_expanded = train

test_labels = test.pop("Labels")

# Scaling features such that they all have a mean of 0 and a variance of 1
scaler = StandardScaler()
scaled_data = scaler.fit_transform(features_X_expanded)

pca = PCA() # can be replaced with "PCA(n_components=2)" but need to check variance ratio first
pca_data = pca.fit_transform(scaled_data)

pca.explained_variance_ratio_ # observing how much each PCA is responsible for the variance

In [19]:
# pca_data_allPCs_df = pd.DataFrame(pca_data, columns = [*range(0, pca_data.shape[1])])

In [ ]:
# extracting all PCs data to cvs file
# pca_data_allPCs_df.to_csv("C:/Users/dnaen/APG_data/pca_data_allPCs.csv", index=False)

In [ ]:
pca = PCA(n_components=2) # to be able to do Euclidean distance calculation set to 2
pca_data_2PCs = pca.fit_transform(scaled_data)
pca_data_2PCs_df = pd.DataFrame(pca_data_2PCs, columns = ['PC1','PC2'])
pca_data_2PCs_df["Labels"] = train_labels # can just append it since row order is not affected

In [ ]:
# setting up test
scaled_test = scaler.transform(test)
pca_test_2PCs = pca.transform(scaled_test)

pca_test_2PCs_df = pd.DataFrame(pca_test_2PCs, columns = ['PC1','PC2'])
pca_test_2PCs_df["Labels"] = test_labels # can just append it since row order is not affected

pca_test_2PCs_df_sorted = pca_test_2PCs_df.sort_values(by=["Labels"])


In [11]:
# to extract file to csv
# pca_data_2PCs_df.to_csv("C:/Users/dnaen/APG_data/pca_data_2PCs.csv", index=False)

# to extract from csv
# pca_data_2PCs_df = pd.read_csv(".../pca_data_2PCs.csv")

In [12]:
# to use at visit prob. calculation later
# pca_data_2PCs_df_sorted = pca_data_2PCs_df.sort_values(by=["Labels"])
# pca_data_2PCs_df_sorted.head(10)

,PC1,PC2,Labels
297926,-2.671005,-0.582621,-1
1392337,-1.088543,-1.647219,-1
1341028,-6.194049,-2.545746,-1
180382,-2.528904,-2.998015,-1
607497,-2.604695,-2.612323,-1
992916,-3.428668,-2.844442,-1
1154659,-4.379559,-1.489158,-1
398164,-2.210939,-2.109961,-1
1102740,-4.489199,1.498144,-1
580377,-2.288275,-2.246409,-1


In [13]:
# experiment 1 - done for 2 PCs
accurate_estimation_prob = 0
false_estimation_prob = 0

total_accurate_prob = 0
total_false_prob = 0

# row size (number of data points)
row_size = len(pca_test_2PCs_df.axes[0])
experiment_size = 2
random_list = [] # for experiment
for y in range(0, experiment_size):
    random_list.append(random.randint(0, row_size))

# only using "pca_data_2comp_df_sorted" such that labels match
for x in random_list:
    label = pca_test_2PCs_df_sorted.at[pca_test_2PCs_df_sorted.index[x], "Labels"]
    if label >= 0:
        current_visit = pca_test_2PCs_df_sorted.iloc[x]
        current_visit = current_visit.tolist()
        current_visit = current_visit[:-1]

        estimated_prob, estimated_label = calculate_prob_of_visit_without_sorting(current_visit, pca_test_2PCs_df_sorted)

        if estimated_label == label:
            total_accurate_prob += estimated_prob
        else:
            total_false_prob += estimated_prob

accurate_estimation_prob = total_accurate_prob / experiment_size
false_estimation_prob = total_false_prob / experiment_size

print("Experiment size:")
print(experiment_size)
print("Accurate prediction with prob.")
print(accurate_estimation_prob)
print("False prediction with prob.")
print(false_estimation_prob)

Experiment size:
2
Accurate prediction with prob.
6.951811433505228e-07
False prediction with prob.
0.0


In [ ]:
"""
Can save results here
"""

In [ ]:
# to use it for next experiment
avg_pca_test_2PCs_df = get_avg_pca(pca_test_2PCs_df)
avg_pca_test_2PCs_df.head()

In [ ]:
# experiment 2 - done for 2 PCs with averaged PCA
accurate_estimation_prob = 0
false_estimation_prob = 0

total_accurate_prob = 0
total_false_prob = 0

# row size (number of data points)
row_size = len(pca_test_2PCs_df_sorted.axes[0])
experiment_size = 100
random_list = [] # for experiment

# for y in range(0, experiment_size):
#    random_list.append(random.randint(0, row_size))

# only using "pca_data_2comp_df_sorted" such that labels match
for x in range(0, row_size):
    label = pca_test_2PCs_df_sorted.at[pca_test_2PCs_df_sorted.index[x], "Labels"]
    if label >= 0:
        current_visit = pca_test_2PCs_df_sorted.iloc[x]
        current_visit = current_visit.tolist()
        current_visit = current_visit[:-1]
        estimated_prob, estimated_label = calculate_prob_of_visit_with_sorted_avg_pca(current_visit, avg_pca_test_2PCs_df)

        if estimated_label == label:
            total_accurate_prob += estimated_prob
        else:
            total_false_prob += estimated_prob

accurate_estimation_prob = total_accurate_prob / experiment_size
false_estimation_prob = total_false_prob / experiment_size

print("Experiment size:")
print(experiment_size)
print("Accurate prediction with prob.")
print(accurate_estimation_prob)
print("False prediction with prob.")
print(false_estimation_prob)

In [ ]:
"""
- without split -
Experiment size:
1438474
Accurate prediction with prob.
0.6983720247985018
False prediction with prob.
0.2959629440643348
"""

In [ ]:
"""
Keeping it just in case
"""
# ----------------------------------------
classifier = DecisionTreeClassifier()
classifier.fit(pca_data, labels_Y) # train classifier

# assuming we got new data
newdata = []

# just transforming to pca, no re-fit again needed
scaled_new_data = scaler.transform(newdata)
pca_new_data = pca.transform(scaled_new_data)

pred_labels = classifier.predict_proba(pca_new_data)

# ---------------------------------------------

# Splitting the dataset into the training set and test set such that it can be used for classification
X_train, X_test, y_train, y_test = train_test_split(features_X, labels_Y, test_size=0.2, random_state=0)

# Scaling features such that they all have a mean of 0 and a variance of 1
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

pca = PCA() # can be replaced with "PCA(n_components=2)" if data is too much
pca_X_train = pca.fit_transform(scaled_X_train)
pca_X_test = pca.transform(scaled_X_test)

pca.explained_variance_ratio_ # observing how much each PCA is responsible for the variance

# Training, Making Predictions and Performance Evaluation
classifier = RandomForestClassifier(max_depth=2, random_state=0)
classifier.fit(pca_X_train, y_train)

# Predicting the test set results and making performance evaluation
y_pred = classifier.predict(pca_X_test)

cm = confusion_matrix(y_test, y_pred)
print("Accuracy:")
print(accuracy_score(y_test, y_pred))
